<a href="https://colab.research.google.com/github/Nomaqui/ENTREGAS/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer

df = pd.read_csv('train.csv')

# Guardar columna objetivo y ID
y = df['RENDIMIENTO_GLOBAL']

X = df.drop(columns=['RENDIMIENTO_GLOBAL', 'ID'])


Separar columnas por tipo

In [2]:
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("Categóricas:", cat_cols)
print("Numéricas:", num_cols)

Categóricas: ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE']
Numéricas: ['PERIODO', 'coef_1', 'coef_2', 'coef_3', 'coef_4']


Imputar numéricas con la media

In [3]:
imp_num = SimpleImputer(strategy='mean')
X[num_cols] = imp_num.fit_transform(X[num_cols])

Imputar categóricas con la moda

In [4]:
imp_cat = SimpleImputer(strategy='most_frequent')
X[cat_cols] = imp_cat.fit_transform(X[cat_cols])

In [5]:
frecuencias = X['ESTU_PRGM_ACADEMICO'].value_counts()

# Crear una máscara para las que aparecen solo una vez
categorias_poco_frecuentes = frecuencias[frecuencias <= 100].index

# Reemplazar esas categorías por "POCO_FRECUENTE"
X['ESTU_PRGM_ACADEMICO'] = X['ESTU_PRGM_ACADEMICO'].replace(categorias_poco_frecuentes, 'POCO_FRECUENTE')

In [6]:
X['ESTU_PRGM_ACADEMICO'].value_counts()

,count
ESTU_PRGM_ACADEMICO,
DERECHO,53244
ADMINISTRACION DE EMPRESAS,51902
CONTADURIA PUBLICA,39664
PSICOLOGIA,31932
INGENIERIA INDUSTRIAL,28481
...,...
LICENCIATURA EN EDUCACI¿N B¿SICA PRIMARIA,103
INGENIERIA DE SISTEMAS Y COMPUTACION,102
COMUNICACION SOCIAL PERIODISMO,102


Mostrar la tabla con los datos imputados

Convertir columnas en one-hot

In [7]:
X_convertida = pd.get_dummies(X, columns=cat_cols, drop_first=True)

Mostrar la tabla con las columnas debidas en one-hot

In [8]:
X_convertida

,PERIODO,coef_1,coef_2,coef_3,coef_4,ESTU_PRGM_ACADEMICO_ADMINISTRACION,ESTU_PRGM_ACADEMICO_ADMINISTRACION AGROPECUARIA,ESTU_PRGM_ACADEMICO_ADMINISTRACION AMBIENTAL,ESTU_PRGM_ACADEMICO_ADMINISTRACION AMBIENTAL Y DE LOS RECURSOS NATURALES,ESTU_PRGM_ACADEMICO_ADMINISTRACION BANCARIA Y FINANCIERA,...,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,20212.0,0.322,0.208,0.310,0.267,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,20212.0,0.311,0.215,0.292,0.264,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,20203.0,0.297,0.214,0.305,0.264,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,20195.0,0.485,0.172,0.252,0.190,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,20212.0,0.316,0.232,0.285,0.294,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,20195.0,0.237,0.271,0.271,0.311,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
692496,20212.0,0.314,0.240,0.278,0.260,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
692497,20183.0,0.286,0.240,0.314,0.287,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
692498,20195.0,0.132,0.426,0.261,0.328,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [26]:
X_convertida['ESTU_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE'].value_counts()

,count
ESTU_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE,
False,692338
True,162


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X_convertida, y,  test_size=0.2, random_state=19, stratify=y)
clf = RandomForestClassifier(
    n_estimators=100,
    #class_weight='balanced',
    # Más árboles suelen mejorar el rendimiento
    max_depth=20,           # Limita profundidad para evitar overfitting
    min_samples_split=5,      # Requiere más muestras para dividir
    min_samples_leaf=2,       # Reduce overfitting
    max_features='sqrt',      # Usa solo raíz de variables en cada split
    random_state=19
)
clf.fit(X_train, y_train)

In [29]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

        alto       0.49      0.62      0.55     35124
        bajo       0.42      0.55      0.47     34597
  medio-alto       0.31      0.20      0.24     34324
  medio-bajo       0.32      0.25      0.28     34455

    accuracy                           0.40    138500
   macro avg       0.38      0.40      0.38    138500
weighted avg       0.38      0.40      0.39    138500

[[21730  4286  5187  3921]
 [ 4467 19084  3988  7058]
 [11237  9076  6696  7315]
 [ 7001 13331  5615  8508]]


In [22]:
df_test = pd.read_csv("test.csv")
ids_test = df_test["ID"]
X_test = df_test.drop(columns=["ID"])

In [23]:
# Imputar valores faltantes
X_test[num_cols] = imp_num.transform(X_test[num_cols])
X_test[cat_cols] = imp_cat.transform(X_test[cat_cols])

# Codificación one-hot con las mismas columnas que el entrenamiento
X_test_convertida = pd.get_dummies(X_test, columns=cat_cols, drop_first=True)

# Asegurar que tenga las mismas columnas que X_convertida
X_test_convertida = X_test_convertida.reindex(columns=X_convertida.columns, fill_value=0)

In [24]:
y_test_pred = clf.predict(X_test_convertida)


In [25]:
print(len(y_test_pred))

296786


In [26]:
submission = pd.DataFrame({
    "ID": ids_test,
    "RENDIMIENTO_GLOBAL": y_test_pred
})

In [27]:
submission.to_csv("submission.csv", index=False)
